# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('Data/city_df.csv')
city_df.dropna(subset=['Lat'], inplace=True)
city_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.4187,19.2345,11.94,79.0,7.0,7.42,ZA,1.661545e+09
1,savonlinna,61.8699,28.8800,14.03,100.0,75.0,0.51,FI,1.661545e+09
2,vaini,-21.2000,-175.2000,25.09,88.0,20.0,3.60,TO,1.661545e+09
3,alofi,-19.0595,-169.9187,24.94,69.0,40.0,6.69,NU,1.661545e+09
4,georgetown,5.4112,100.3354,27.96,87.0,20.0,2.06,MY,1.661545e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key= g_key)
locations = city_df[['Lat', 'Lng']]
weights = city_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness
vacation_locations = city_df[city_df['Max Temp'] > 20]
vacation_locations = vacation_locations[vacation_locations['Max Temp'] < 30]
vacation_locations = vacation_locations[vacation_locations['Wind Speed'] < 15]
vacation_locations = vacation_locations[vacation_locations['Cloudiness'] == 0]
vacation_locations.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,komsomolskiy,40.4272,71.7189,20.40,32.0,0.0,2.06,UZ,1.661545e+09
19,jalu,29.0331,21.5482,28.51,37.0,0.0,4.74,LY,1.661545e+09
45,antsohihy,-14.8796,47.9875,20.50,56.0,0.0,4.22,MG,1.661546e+09
53,vaitape,-16.5167,-151.7500,25.11,69.0,0.0,2.95,PF,1.661546e+09
60,kapaa,22.0752,-159.3190,28.39,72.0,0.0,4.63,US,1.661546e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
ideal_cities = vacation_locations.City.unique()
hotel_df = vacation_locations
hotel_df = hotel_df.set_index('City')
hotel_df['Hotel Name'] = ''
hotel_df.head(5)

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City,,,,,,,,,
komsomolskiy,40.4272,71.7189,20.40,32.0,0.0,2.06,UZ,1.661545e+09,
jalu,29.0331,21.5482,28.51,37.0,0.0,4.74,LY,1.661545e+09,
antsohihy,-14.8796,47.9875,20.50,56.0,0.0,4.22,MG,1.661546e+09,
vaitape,-16.5167,-151.7500,25.11,69.0,0.0,2.95,PF,1.661546e+09,
kapaa,22.0752,-159.3190,28.39,72.0,0.0,4.63,US,1.661546e+09,


In [57]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
for c in ideal_cities:
    params = {'location':f'{hotel_df.loc[c].Lat}, {hotel_df.loc[c].Lng}',
             'radius':1500,
             'type':'hotel',
             'key':g_key}
    try:
        hotel_df.at[c, 'Hotel Name'] = requests.get(base_url, params = params).json()['results'][1]['name']
    except:
        print('Unable to get str(c)')

hotel_df = hotel_df.reset_index(level=0)
hotel_df.head(5)
        
        

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City,,,,,,,,,
komsomolskiy,40.4272,71.7189,20.40,32.0,0.0,2.06,UZ,1.661545e+09,Mashal Koʻchasi 42A
jalu,29.0331,21.5482,28.51,37.0,0.0,4.74,LY,1.661545e+09,شركة تكساس لشحن وإستيراد السيارات
antsohihy,-14.8796,47.9875,20.50,56.0,0.0,4.22,MG,1.661546e+09,Agence BNI Madagascar
vaitape,-16.5167,-151.7500,25.11,69.0,0.0,2.95,PF,1.661546e+09,Haumana Cruise Bora Bora to Taha'a
kapaa,22.0752,-159.3190,28.39,72.0,0.0,4.63,US,1.661546e+09,Hotel Coral Reef


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [65]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(city_df[['Lat', 'Lng']], weights= city_df['Humidity']))
fig.add_layer(gmaps.marker_layer(hotel_df[["Lat", "Lng"]], info_box_content=hotel_info, hover_text = hotel_df.City))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))